In [2]:
import numpy as np
import pandas as pd

In [3]:
election_data_path = '../../datasets/Raw Datasets/Electoral Data/'
census_data_path = '../../datasets/Raw Datasets/Census Data/'
other_data_path = '../../datasets/Raw Datasets/Other Data/'

# Electorate changes

In [4]:
ced_2016 = pd.read_csv(other_data_path + 'CED_2016_AUST.csv')[['SA1_MAINCODE_2016', 'CED_NAME_2016']].set_index('SA1_MAINCODE_2016')
ced_2018 = pd.read_csv(other_data_path + 'CED_2018_AUST.csv')[['SA1_MAINCODE_2016', 'CED_NAME_2018']].set_index('SA1_MAINCODE_2016')

In [7]:
ced_to_from = ced_2016.join(ced_2018)
ced_to_from = ced_to_from[ced_to_from['CED_NAME_2016'] != ced_to_from['CED_NAME_2018']].sort_index()
ced_to_from = ced_to_from.reset_index()
ced_to_from['SA1_CODE'] = ced_to_from['SA1_MAINCODE_2016'].transform(lambda x: str(x)[0] + str(x)[-6:])
ced_to_from = ced_to_from.set_index('SA1_CODE')
ced_to_from.drop('SA1_MAINCODE_2016', axis=1, inplace=True)
display(len(set(ced_to_from['CED_NAME_2016']).union(set(ced_to_from['CED_NAME_2018']))))
ced_to_from

84

,CED_NAME_2016,CED_NAME_2018
SA1_CODE,,
2100701,Ballarat,Wannon
2100702,Ballarat,Wannon
2100703,Ballarat,Wannon
2100704,Ballarat,Wannon
2100705,Ballarat,Wannon
...,...,...
9100404,Canberra,Bean
9100405,Canberra,Bean
9100406,Canberra,Bean


In [5]:
removed_electorates = set(ced_2016['CED_NAME_2016']) - set(ced_2018['CED_NAME_2018'])
new_electorates = set(ced_2018['CED_NAME_2018']) - set(ced_2016['CED_NAME_2016'])

display(removed_electorates)
display(new_electorates)

{'Batman',
 'Denison',
 'McMillan',
 'Melbourne Ports',
 'Murray',
 'Port Adelaide',
 'Wakefield'}

{'Bean',
 'Clark',
 'Cooper',
 'Fraser',
 'Macnamara',
 'Monash',
 'Nicholls',
 'Spence'}

In [6]:
ced_to_from[ced_to_from['CED_NAME_2016'].isin(removed_electorates)].drop_duplicates().reset_index(drop=True).groupby('CED_NAME_2016').count()

,CED_NAME_2018
CED_NAME_2016,
Batman,3
Denison,1
McMillan,3
Melbourne Ports,1
Murray,3
Port Adelaide,4
Wakefield,3


In [7]:
import os

ced_files = []
sa1_files = []
ced_count = 0
sa1_count = 0

for path in os.scandir(census_data_path):
    if path.is_file():
        print(path.name)
        
        if path.name[0:3] == 'CED':
            ced_files.append(path.path)
        elif path.name[0:3] == 'SA1':
            sa1_files.append(path.path)

assert(len(ced_files) == len(sa1_files))

CED to Age (Individual years).csv
CED to Ages (5 year brackets).csv
CED to Education level.csv
CED to Income.csv
CED to Job (Industry).csv
CED to Labour Force Status.csv
CED to Public-Private Sector.csv
CED to Registered Marital Status.csv
CED to Relationships in Households.csv
CED to Religion.csv
CED to Sex.csv
CED to Social Marital Status.csv
SA1 to Age (Individual years).csv
SA1 to Ages (5 year brackets).csv
SA1 to Education level.csv
SA1 to Income.csv
SA1 to Job (Industry).csv
SA1 to Labour Force Status.csv
SA1 to Public-Private Sector.csv
SA1 to Registered Marital Status.csv
SA1 to Relationships in Households.csv
SA1 to Religion.csv
SA1 to Sex.csv
SA1 to Social Marital Status.csv


# Cleaning tests

In [8]:
data_suffix = 'Social Marital Status'

In [9]:
sa1_data = pd.read_csv(census_data_path + 'SA1 to ' + data_suffix + '.csv', engine='python', skiprows=9, skipfooter=9).drop(0).iloc[:,:-2]
sa1_data.set_index(sa1_data.columns[0], inplace=True)
sa1_data

,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
MDCP Social Marital Status,,,,
1102801,0.0,0.0,0.0,0.0
1102802,246.0,57.0,143.0,163.0
1102803,237.0,24.0,129.0,230.0
1102804,89.0,5.0,87.0,56.0
1102805,108.0,31.0,85.0,106.0
...,...,...,...,...
9100406,0.0,0.0,0.0,0.0
9100407,147.0,38.0,80.0,107.0
9100408,0.0,0.0,0.0,0.0


In [10]:
ced_to_from_data = ced_to_from.join(sa1_data)
ced_to_from_data

,CED_NAME_2016,CED_NAME_2018,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
SA1_CODE,,,,,,
2100701,Ballarat,Wannon,203.0,36.0,94.0,151.0
2100702,Ballarat,Wannon,157.0,12.0,85.0,76.0
2100703,Ballarat,Wannon,189.0,33.0,91.0,148.0
2100704,Ballarat,Wannon,184.0,31.0,96.0,94.0
2100705,Ballarat,Wannon,166.0,41.0,99.0,130.0
...,...,...,...,...,...,...
9100404,Canberra,Bean,86.0,44.0,89.0,74.0
9100405,Canberra,Bean,0.0,0.0,0.0,0.0
9100406,Canberra,Bean,0.0,0.0,0.0,0.0


In [11]:
ced_to_from_data.isna().sum()

CED_NAME_2016                       0
CED_NAME_2018                       0
Married in a registered marriage    0
Married in a de facto marriage      0
Not married                         0
Not applicable                      0
dtype: int64

In [12]:
ced_to_from_group_data = ced_to_from_data.groupby(['CED_NAME_2016', 'CED_NAME_2018']).sum().reset_index()
ced_to_from_group_data

,CED_NAME_2016,CED_NAME_2018,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
0,Adelaide,Boothby,2187.0,486.0,2117.0,1443.0
1,Adelaide,Sturt,8291.0,1769.0,8071.0,5945.0
2,Ballarat,Wannon,2336.0,449.0,1252.0,1630.0
3,Bass,Lyons,3322.0,658.0,2491.0,2385.0
4,Batman,Cooper,45743.0,14398.0,54387.0,39602.0
...,...,...,...,...,...,...
90,Wannon,Mallee,5270.0,1208.0,5192.0,4045.0
91,Wide Bay,Hinkler,484.0,132.0,367.0,425.0
92,Wills,Cooper,1193.0,422.0,1266.0,1054.0
93,Wills,Maribyrnong,2808.0,460.0,1907.0,1858.0


In [13]:
# e.g. Port Adelaide was dissolved
ced_to_from_data[ced_to_from_data['CED_NAME_2016'] == 'Port Adelaide'].groupby(['CED_NAME_2016', 'CED_NAME_2018']).sum().reset_index()

,CED_NAME_2016,CED_NAME_2018,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
0,Port Adelaide,Adelaide,6995.0,1304.0,7970.0,5711.0
1,Port Adelaide,Hindmarsh,25201.0,6932.0,29622.0,19510.0
2,Port Adelaide,Makin,6216.0,1039.0,5743.0,4561.0
3,Port Adelaide,Spence,13155.0,3509.0,15040.0,12487.0


In [14]:
ced_data = pd.read_csv(census_data_path + 'CED to ' + data_suffix + '.csv', engine='python', skiprows=9, skipfooter=9).drop(0).iloc[:,:-2]
ced_data

,MDCP Social Marital Status,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
1,Banks,60121.0,6983.0,52229.0,36469.0
2,Barton,61655.0,10607.0,59824.0,40771.0
3,Bennelong,66619.0,8087.0,54464.0,39783.0
4,Berowra,64395.0,5216.0,39070.0,36449.0
5,Blaxland,59962.0,5055.0,60475.0,49140.0
...,...,...,...,...,...
164,Fenner,69285.0,17268.0,58560.0,57838.0
165,No usual address (ACT),0.0,0.0,0.0,541.0
166,Migratory - Offshore - Shipping (ACT),0.0,0.0,0.0,0.0
167,No usual address (OT),0.0,0.0,0.0,54.0


In [15]:
ced_to_from_group_data[ced_to_from_group_data['CED_NAME_2016'] == 'Melbourne Ports']

,CED_NAME_2016,CED_NAME_2018,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
73,Melbourne Ports,Macnamara,41691.0,20409.0,57645.0,41046.0


In [16]:
ced_to_from_group_data[ced_to_from_group_data['CED_NAME_2016'] == 'Denison'].drop(['CED_NAME_2016', 'CED_NAME_2018'], axis=1).sum()

Married in a registered marriage    29887.0
Married in a de facto marriage       9454.0
Not married                         35181.0
Not applicable                      28007.0
dtype: float64

In [17]:
ced_data[ced_data.iloc[:,0] == 'Aston']

,MDCP Social Marital Status,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
50,Aston,54647.0,7735.0,41510.0,32129.0


In [18]:
# Totals that need to be subtracted from electorates
rem = ced_to_from_group_data.drop('CED_NAME_2018', axis=1).groupby('CED_NAME_2016').sum()
rem

,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
CED_NAME_2016,,,,
Adelaide,10478.0,2255.0,10188.0,7388.0
Ballarat,2336.0,449.0,1252.0,1630.0
Bass,3322.0,658.0,2491.0,2385.0
Batman,48413.0,14901.0,57297.0,41243.0
Bendigo,1809.0,296.0,972.0,1297.0
Boothby,13893.0,2301.0,8383.0,8120.0
Brisbane,1219.0,409.0,1292.0,1275.0
Bruce,36454.0,2768.0,27162.0,19673.0
Calwell,18016.0,1923.0,13741.0,10265.0


In [19]:
# Totals that need to be added to electorates
ad = ced_to_from_group_data.drop('CED_NAME_2016', axis=1).groupby('CED_NAME_2018').sum()
ad

,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
CED_NAME_2018,,,,
Adelaide,14815.0,3295.0,17951.0,11689.0
Aston,6072.0,1629.0,5556.0,4851.0
Barker,6382.0,1317.0,3889.0,4496.0
Bass,3854.0,786.0,2489.0,2622.0
Bean,49815.0,10563.0,38561.0,36875.0
Blair,2582.0,443.0,1329.0,1746.0
Boothby,16751.0,4202.0,19005.0,12307.0
Braddon,2329.0,487.0,1400.0,1630.0
Bruce,39979.0,4636.0,36376.0,26479.0


In [20]:
ced_data_clean = ced_data.set_index(ced_data.columns[0])\
    .sub(rem, fill_value=0)\
    .add(ad, fill_value=0)\
    .drop(removed_electorates)\
    .astype(int)
ced_data_clean

,Married in a registered marriage,Married in a de facto marriage,Not married,Not applicable
Adelaide,54682,13346,65853,46993
Aston,60719,9364,47066,36980
Ballarat,48120,12179,44914,43598
Banks,60121,6983,52229,36469
Barker,59520,12946,45347,47779
...,...,...,...,...
Werriwa,61308,6746,53633,49213
Whitlam,55755,10291,44845,41384
Wide Bay,50070,11441,40546,40633
Wills,47097,14495,55920,37378


In [21]:
ced_data_clean.loc['Bass']

Married in a registered marriage    33016
Married in a de facto marriage       8376
Not married                         31313
Not applicable                      29581
Name: Bass, dtype: int32

In [22]:
ced_data.set_index(ced_data.columns[0]).sort_index().loc['Bass']

Married in a registered marriage    32484.0
Married in a de facto marriage       8248.0
Not married                         31315.0
Not applicable                      29344.0
Name: Bass, dtype: float64

In [ ]:
ced_data_clean.index.rename(ced_data.columns[0], inplace=True)
ced_data_clean.to_csv(census_data_path + 'clean/' + 'CED to ' + data_suffix + '.csv')